In [40]:
import numpy as np
import pandas as pd
import json

raw = pd.read_csv('311withbbls.csv.tar.gz', compression='gzip',
                   usecols = ['Created Date', 'Latitude', 'Longitude', 'bbl', 'Complaint Type'])
raw.head()

,Created Date,Complaint Type,Latitude,Longitude,bbl
0,02/17/2014 05:33:24 PM,General Construction/Plumbing,40.833111,-73.869561,2099998900
1,02/17/2014 05:27:59 PM,General Construction/Plumbing,40.836302,-73.852812,2099998900
2,02/17/2014 05:27:00 PM,Snow,40.827145,-73.847961,2099998900
3,02/17/2014 05:25:24 PM,Blocked Driveway,40.845491,-73.852578,2099998900
4,02/17/2014 05:22:44 PM,Building/Use,40.856630,-73.840563,2042220046


In [41]:
# Take only the year portion of "Created Date"

data = raw.rename(columns = {'Created Date': 'Year'})
data['Year'] = data.apply(lambda x: np.int64(x.Year[6:10]), axis = 1)
data.dropna(inplace = True)
data.head()


,Year,Complaint Type,Latitude,Longitude,bbl
0,2014,General Construction/Plumbing,40.833111,-73.869561,2099998900
1,2014,General Construction/Plumbing,40.836302,-73.852812,2099998900
2,2014,Snow,40.827145,-73.847961,2099998900
3,2014,Blocked Driveway,40.845491,-73.852578,2099998900
4,2014,Building/Use,40.856630,-73.840563,2042220046


In [46]:
# Group together and count complaints of the same type from the same BBL in the same year

groupby_obj = data[['Year', 'Complaint Type', 'Latitude','Longitude','bbl']].groupby(['Year', 'Complaint Type', 'bbl'])
counted = groupby_obj.agg('mean')\
    .join(pd.DataFrame(groupby_obj.size(), columns=['Complaints Count']))\
    .reset_index()
counted['Year'] = counted['Year'].astype(np.int64)
counted['bbl'] = counted['bbl'].astype(np.int64)
counted.head()

,Year,Complaint Type,bbl,Latitude,Longitude,Complaints Count
0,2010,APPLIANCE,2000008900,40.854424,-73.908837,270
1,2010,APPLIANCE,2022670044,40.807918,-73.917972,1
2,2010,APPLIANCE,2022690001,40.810550,-73.917262,5
3,2010,APPLIANCE,2022690021,40.809934,-73.916369,1
4,2010,APPLIANCE,2022710005,40.812113,-73.915897,1


In [47]:
print (counted.shape)
print (counted.describe())

# counted.to_csv('ComplaintCountByBBL.csv', index = False)


(133422, 6)
                Year           bbl       Latitude      Longitude  \
count  133422.000000  1.334220e+05  133422.000000  133422.000000   
mean     2012.619501  2.034584e+09      40.849347     -73.890990   
std         1.745382  1.165353e+07       0.024941       0.028864   
min      2010.000000  2.000009e+09      40.788286     -73.933546   
25%      2011.000000  2.026980e+09      40.830371     -73.912566   
50%      2013.000000  2.031320e+09      40.847138     -73.898344   
75%      2014.000000  2.036810e+09      40.868811     -73.880914   
max      2015.000000  2.099999e+09      40.912572     -73.782065   

       Complaints Count  
count     133422.000000  
mean           7.515237  
std           77.393029  
min            1.000000  
25%            1.000000  
50%            2.000000  
75%            4.000000  
max         8459.000000  


In [48]:
output = {}

for year in np.unique(counted['Year'].values):
    output[year] = []
    hold = counted[counted.Year == year]
    for index,row in hold.iterrows():
        entry = {}
        entry['bbl'] = row['bbl']
        entry['latitude'] = row['Latitude']
        entry['longitutde'] = row['Longitude']
        entry['complaint_count'] = row['Complaints Count']
        entry['complaint_type'] = row['Complaint Type']
        output[year].append(entry)
        
with open('complaintByBBL.json', 'w') as f:
    json.dump(output, f)

In [ ]:
print (output['2010'])